# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846826225397                   -0.70    4.5
  2   -7.852305930804       -2.26       -1.53    1.0
  3   -7.852645965100       -3.47       -2.53    3.0
  4   -7.852646676977       -6.15       -3.36    2.2
  5   -7.852646685819       -8.05       -4.79    1.2
  6   -7.852646686723       -9.04       -5.16    4.5
  7   -7.852646686729      -11.20       -5.89    2.0
  8   -7.852646686730      -12.44       -7.38    2.0
  9   -7.852646686730      -14.75       -7.87    2.5
 10   -7.852646686730   +    -Inf       -8.86    1.5
 11   -7.852646686730      -14.75       -9.27    1.0
 12   -7.852646686730   +    -Inf      -10.25    2.5
 13   -7.852646686730      -14.75      -10.67    1.0
 14   -7.852646686730   +  -14.57      -11.97    2.0
 15   -7.852646686730   +  -15.05      -12.38    2.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846839869176                   -0.70           4.8
  2   -7.852526705316       -2.25       -1.63   0.80    2.0
  3   -7.852637204886       -3.96       -2.71   0.80    1.0
  4   -7.852646422748       -5.04       -3.27   0.80    2.0
  5   -7.852646665170       -6.62       -4.07   0.80    1.2
  6   -7.852646686369       -7.67       -4.68   0.80    2.2
  7   -7.852646686725       -9.45       -5.39   0.80    1.5
  8   -7.852646686729      -11.41       -6.73   0.80    1.2
  9   -7.852646686730      -12.16       -6.89   0.80    3.0
 10   -7.852646686730      -14.75       -7.86   0.80    1.0
 11   -7.852646686730      -14.57       -9.08   0.80    1.5
 12   -7.852646686730   +  -14.57       -9.01   0.80    2.5
 13   -7.852646686730   +  -15.05      -10.33   0.80    1.0
 14   -7.852646686730   +  -14.75      -10.95   0.80    2.0
 15   -7.852646686730      -14.57      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.365968e+01     3.280975e+00
 * time: 0.5807921886444092
     1     1.183809e+00     1.839616e+00
 * time: 0.8456530570983887
     2    -1.422765e+00     2.149561e+00
 * time: 0.8742411136627197
     3    -3.666891e+00     1.852976e+00
 * time: 0.916438102722168
     4    -4.867372e+00     1.697373e+00
 * time: 0.9589550495147705
     5    -6.587807e+00     9.212788e-01
 * time: 1.0013110637664795
     6    -7.261283e+00     4.489107e-01
 * time: 1.050441026687622
     7    -7.543391e+00     3.213930e-01
 * time: 1.0795001983642578
     8    -7.642488e+00     1.568896e-01
 * time: 1.109166145324707
     9    -7.729159e+00     1.762318e-01
 * time: 1.1385111808776855
    10    -7.785327e+00     1.156709e-01
 * time: 1.1693191528320312
    11    -7.826537e+00     6.111392e-02
 * time: 1.198876142501831
    12    -7.845058e+00     5.086118e-02
 * time: 1.230790138244629
    13    -7.850605e+00     2.253899e-02
 * time: 1.262890100479126

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846906459098                   -0.70    4.8
  2   -7.852323988187       -2.27       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686714                   -2.55
  2   -7.852646686730      -10.80       -6.04
  3   -7.852646686730   +  -15.05      -12.71


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 9.402366497858104e-14
|ρ_newton - ρ_scfv| = 2.645696291891693e-13
|ρ_newton - ρ_dm|   = 8.801517977463509e-10
